# Example of Deduplication

In [7]:
import pandas as pd
from duplicatesuricate import deduplication as dep
from duplicatesuricate.preprocessing import clean_db

## 1. Loading Data

### Loading sample data

In [8]:
df_input_records = pd.read_csv('df_impairs.csv',index_col=0,dtype={'duns':str,'postalcode':str},sep='|')
df_target_records = pd.read_csv('df_pairs.csv',index_col=0,dtype={'duns':str,'postalcode':str},sep='|')
df_input_records.sample(3)

,name,duns,city,postalcode,street,country_code
gid,,,,,,
76e834db-6e0c-400a-8f51-98b2652fd479,datalis,260332779,portet-sur-garonne,31125,8 avenue gutenberg,FR
5b73dbc1-e030-4d16-a8ce-3935c6094aa7,goodrich sensorsintegrated syst,804817070,burnsville,55306-4890,14300 judicial rd,US
e442ea3c-895b-44b2-bdf2-bda0aa36bf18,horizon platforms limited,210161671,normanton,wf6 1zb,rosie road,GB


### Cleaning that data

In [9]:
for x in [df_input_records,df_target_records]:
    x = clean_db(x)
df_target_records.sample(3)

,name,duns,city,postalcode,street,country_code,name_wostopwords,street_wostopwords,name_acronym,postalcode_1stdigit,postalcode_2digits,name_len,hasairbusname,isbigcity
gid,,,,,,,,,,,,,,
efd9b1bc-6310-4eb9-8d6f-42c4d8375166,goetz service gmbh,None,goppingen,73037,2 2 zillenhardtstr,DE,goetz,2 zillenhardt,gsg,7,73,18,0,0
546c2c52-71cd-4880-ad20-e0e9a0ff2cc6,licatec gmbh,330944281,brand erbisdorf,9618,7 erzstr,DE,licatec,7 erz,lg,9,96,12,0,0
875615ec-9b8a-4a46-a4b9-a7b1d0bc08bc,me systeme,None,kolleda,99625,2 august bebel str,DE,me systeme,bebel 2 august,ms,9,99,10,0,0


# Machine Learning - based deduplication

## 1. Training set

### Option 1: Creating a training table for the decision model
#### Step 1 Loading a supervised learning table

In [10]:
km=pd.read_csv('known_matches3.csv')
rowmax=None
print('nrows: {}'.format(km.shape[0]))
km = km.loc[km['ix_source'].isin(df_input_records.index)]
km = km.loc[km['ix_target'].isin(df_target_records.index)]
print('nrows: {}'.format(km.shape[0]))
y=km['y_true'].value_counts()
print('Positive labels:{0:.0%}'.format(y[1]/y.sum()))
del y
km.sample(3)

nrows: 3770
nrows: 43
Positive labels:100%


,ix_source,ix_target,y_proba,y_true
1034,ea502496-0ffc-4d51-8ea1-8e4789ac3a1e,0bcf1f54-8923-4e65-9012-3de0592a4509,1.0000,1.0
1118,dcecaed1-df61-4e48-84e3-f990e5b78974,5579240e-0518-4b54-9f1a-d04a89569355,0.9995,1.0
986,b37479a6-1be0-435f-8d93-08e98199e844,9d90ddbc-534f-4edd-91b9-a4acd9f6d751,1.0000,1.0


#### Step 2 Creating the training table

In [11]:
dummymodel=dep.DummyClassifier(scoredict={'fuzzy':['name','name_wostopwords',
                                               'street','street_wostopwords',
                                               'city'],
                                      'exact':['duns','country_code'],
                                      'token':['name','name_wostopwords',
                                               'street','street_wostopwords'],
                                      'acronym':['name','name_wostopwords']})
sur=dep.Suricate(input_records=df_input_records,
                target_records=df_target_records,
                classifier=dummymodel)
training_table=sur.build_training_table(inputs=km['ix_source'].values,
                                        targets=km['ix_target'].values,
                                        y_true=km['y_true'].values,
                                       with_proba=False).fillna(0)
#x2=sur.chain_build_labelled_table(inputs=s_inputs,targets=s_targets)
print(training_table['y_true'].value_counts())
training_table.sample(3)

1.0    43
Name: y_true, dtype: int64


,,city_fuzzyscore,country_code_exactscore,duns_exactscore,name_acronymscore,name_fuzzyscore,name_tokenscore,name_wostopwords_acronymscore,name_wostopwords_fuzzyscore,name_wostopwords_tokenscore,street_fuzzyscore,street_tokenscore,street_wostopwords_fuzzyscore,street_wostopwords_tokenscore,y_true
ix_source,ix_target,,,,,,,,,,,,,,
e442ea3c-895b-44b2-bdf2-bda0aa36bf18,716a90f4-0066-4e26-a727-9575802f9ffd,1.0,1.0,0.0,-0.52,-0.02,1.0,0.00,-0.02,1.0,1.0,1.0,1.0,1.0,1.0
21c09dde-cff3-4c45-a2f1-46a99e6e1587,1b932c6a-3719-4f78-ba6e-c4ffdf0cc344,1.0,1.0,0.0,-0.64,1.00,1.0,0.00,1.00,1.0,1.0,1.0,1.0,1.0,1.0
424803ef-a428-46eb-999b-d68429aedf5d,e974ef74-3bf5-45c2-9273-c44f73945a4d,1.0,1.0,0.0,-0.64,1.00,1.0,-0.64,1.00,1.0,1.0,1.0,1.0,1.0,1.0


### Or Option 2: Loading an existing table

In [12]:
training_table=pd.read_csv('trainining_table3.csv',sep=',',index_col=[0,1])
training_table.sample(3)

,,city_fuzzyscore,country_code_exactscore,duns_exactscore,hasairbusname_source,hasairbusname_target,isbigcity_source,isbigcity_target,name_acronym_fuzzyscore,name_acronymscore,name_fuzzyscore,name_tokenscore,name_wostopwords_fuzzyscore,name_wostopwords_tokenscore,postalcode_1stdigit_exactscore,postalcode_2digits_exactscore,postalcode_exactscore,postalcode_fuzzyscore,street_fuzzyscore,street_tokenscore,street_wostopwords_fuzzyscore,street_wostopwords_tokenscore,y_true
ix_source,ix_target,,,,,,,,,,,,,,,,,,,,,,
3b79b4b8-04f1-4198-8dab-9abd5c3b1e2b,aed69ed2-8438-41b5-a7f1-47046dae5981,1.0,1.0,0.0,0,0,1,1,0.72,1.00,0.86,1.00,0.86,1.00,0.0,0.0,0.0,-0.14,-0.52,-0.42,-0.58,-0.44,1.0
dcc308de-62f6-4c91-bd0e-b7ad77424c9c,c14c6e12-efc9-486d-9b89-d8b50be8e36c,1.0,1.0,0.0,0,0,0,0,1.00,-0.52,1.00,1.00,1.00,1.00,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.0
0472d735-b408-4d20-9209-858c5ac35232,b79a078a-156f-421b-934f-25da716214cc,-1.0,1.0,0.0,0,0,0,0,0.34,0.00,-0.46,-0.46,-0.46,-0.46,1.0,1.0,0.0,0.20,-0.70,1.00,-0.62,1.00,0.0


## Use a Scikit-Learn Classifier
Go to the end of the notebook to see how you can use a SparkML classifier or a Rule-based classifier

optional : select yourself the evaluator you need

In [13]:
from sklearn.ensemble import RandomForestClassifier
mymodel = RandomForestClassifier(n_estimators=1000,max_depth=30,min_samples_split=3)
classifier=dep.ScikitLearnClassifier(model=mymodel)

or use the default one (RandomForest)

In [14]:
classifier=dep.ScikitLearnClassifier()

fit

In [15]:
X_train = training_table.iloc[:,:-1].astype(float)
y_train= training_table.iloc[:,-1].astype(int)
classifier.fit(X=X_train,y=y_train)

shape of training table  (3770, 21)
number of positives in table 1389
precision score on training data: 0.987883107627
recall score on training data: 0.997840172786
time elapsed 7.686195 seconds


### Optional: add filtering rules to speed up the scoring process

filter the target records on records that match exactly the country code, or that match the duns number of the query

In [16]:
filterdict={'all':['country_code'],
           'any':['duns']}

from those filtered records, filter on records who have a roughly similar name or address, or share the same duns

In [17]:
intermediate_thresholds={'name_wostopwords_fuzzyscore':0.5,'street_wostopwords_fuzzyscore':0.5, 'duns_exactscore':1.0,'aggfunc':'any'}

putting all together

In [18]:
sur=dep.Suricate(input_records=df_input_records,target_records=df_target_records,
                 filterdict=filterdict,
                 intermediate_thresholds=intermediate_thresholds,
                 classifier=classifier)

# Launching the deduplication

### Possibility 1: return only good matches (for run mode)

In [19]:
res=sur.start_linkage(n_matches_max=None,with_proba=False)
x=sur.build_visualcomparison_table(inputs=res['ix_source'].values,targets=res['ix_target'].values,display=['name',],fuzzy=['street'],exact=['duns'])
x.sample(3)

starting deduplication at 2018-02-13 16:28:17.845890
1 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.293408 s
2 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.33683 s
3 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.248936 s
4 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.251387 s
5 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.284098 s
6 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.299354 s
7 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.26093 s
8 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.317409 s
9 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.308501 s
10 of 10 inputs records deduplicated | f

,,name_source,name_target,duns_source,duns_target,duns_exactscore,street_source,street_target,street_fuzzyscore
ix_source,ix_target,,,,,,,,
31ea5edd-2c80-40ca-85ca-b6768a941d1e,7ff2b1d2-bf47-4b26-849a-0d150fca7b66,h media,h media,372377817,None,0.0,329 heistraat,329 heistraat,1.0
e2a2da69-3aa4-44e2-ae5b-4bbd2cbdc238,c906f2e3-bd4c-4785-9d60-95f19579a04c,botschaft afghanistan,botschaft afghanistan,None,None,0.0,3 taunusstraaye,3 taunusstr,0.7
5ff704ee-399e-4fbd-b604-51b6ced944dd,af8133f8-361f-494e-92dc-ab3c72637d56,berlinzeppelin,berlinzeppelin,None,None,0.0,4 rottweiler str,4 rottweiler str,1.0


### Possibility 2: return a probability vector to build a supervised learning table
this will give you the  {n_matches_max} most probable records, and thus help to train the tool with records on the decision frontier

In [20]:
# return the 5 most probable matches of the query and the associated probabilities
res=sur.start_linkage(n_matches_max=5,with_proba=True)

starting deduplication at 2018-02-13 16:28:20.843730
1 of 10 inputs records deduplicated | found 1 of 5 max possible matches | time elapsed 0.314011 s
2 of 10 inputs records deduplicated | found 1 of 5 max possible matches | time elapsed 0.196651 s
3 of 10 inputs records deduplicated | found 1 of 5 max possible matches | time elapsed 0.340603 s
4 of 10 inputs records deduplicated | found 1 of 5 max possible matches | time elapsed 0.349558 s
5 of 10 inputs records deduplicated | found 1 of 5 max possible matches | time elapsed 0.374056 s
6 of 10 inputs records deduplicated | found 1 of 5 max possible matches | time elapsed 0.416584 s
7 of 10 inputs records deduplicated | found 1 of 5 max possible matches | time elapsed 0.329333 s
8 of 10 inputs records deduplicated | found 1 of 5 max possible matches | time elapsed 0.229988 s
9 of 10 inputs records deduplicated | found 1 of 5 max possible matches | time elapsed 0.238179 s
10 of 10 inputs records deduplicated | found 1 of 5 max possible 

In [21]:
x=sur.build_visualcomparison_table(inputs=res['ix_source'].values,targets=res['ix_target'].values,display=['name',],fuzzy=['street'],exact=['duns'],
                                   y_proba=res['y_proba'])
x.sort_values(by='y_proba',ascending=True).head(3)

,,name_source,name_target,duns_source,duns_target,duns_exactscore,street_source,street_target,street_fuzzyscore,y_proba
ix_source,ix_target,,,,,,,,,
c204c7b7-66fd-4e55-99a4-abf647dd6b3c,7a17d0c9-5992-43db-9580-c1c6cf16cdbc,l amphitryon restaurant,alter ego 31,779097252,779097252,1.0,chemin de gramont,chemin de gramont,1.00,0.9540
f3bfed60-958f-454d-8aeb-8674989f7b35,fa96a218-64c7-48b5-8156-2023308ba14e,vds,vds schadenverhtung gmbh,None,332836506,0.0,29 rheinstr,rheinstr,0.68,0.9800
4de8d106-40f7-45cc-a0ac-27fbfe94d7fb,0f8eb8df-0881-4d88-af56-b1b75421cd4c,ucon ag,ucon ag containersysteme kg,341829459,341057797,-1.0,122 sechsheldener str,sechsheldener str,0.78,0.9825


# More possibilites: other kind of classifiers
----

## Rule-based classifier
In place of the model learning from training data, you hard-code some rules.  
It is useful if you want to have a predictable and explainable model.  

In [22]:
# example:
hard_threshold = {'name_tokenscore': 0.7,
                  'street_tokenscore': 0.7}
# this hard coded function will take as input r : the row of the scoring
# if one one the score for the row is below the threshold, it will return 0
# you can
def hardcodedfunc(r):
    r = r.fillna(0)
    for k in hard_threshold.keys():
        if r[k] < hard_threshold[k]:
            return 0
    else:
        return 1
# What is important is to note all the columns needed for scoring so the tool knows which score to calculate
score_cols = list(hard_threshold.keys())

In [23]:
classifier = dep.RuleBasedClassifier(used_cols=score_cols,
                        eval_func=hardcodedfunc)

## Spark it! Using a SparkML Model
The bigger the target records dataset is, the more calculations there are to perform, the slower this algorithm goes.  
One crude solution is to throw more computers at the problem: using Spark for parallel computing.

#### With an sqlContext
do not forget to update the spark context if the name of the module if necessary

In [24]:
import findspark,pyspark
from pyspark.sql import SQLContext
#mypath='/Users/paulogier/Documents/8-PythonProjects/duplicatesuricate/duplicatesuricate/'
sc = pyspark.SparkContext(appName="DuplicateSuricate")
sqlCtx = SQLContext(sc)

#### Init and fit classifier.
Fit is done sklearn-style with model.fit(X,y) (and accepts dataframe as inputs) for consistency with the other classifiers

In [25]:
classifier = dep.SparkClassifier(sqlContext=sqlCtx)
classifier.fit(X_train,y_train)

shape of training table  (3770, 21)
number of positives in table 1389
precision score on training data: 0.947513812155
recall score on training data: 0.987760979122
time elapsed 14.149353 seconds


#### Init the Suricate interface with the sparkML
there is an additional argument spark = True to let the classifier know that you have initialized an sqlContext in the evaluator

In [26]:
sur = dep.Suricate(input_records=df_input_records,target_records=df_target_records,
                 filterdict=filterdict,
                 intermediate_thresholds=intermediate_thresholds,
                 classifier=classifier,spark = True)

In [27]:
res=sur.start_linkage(n_matches_max=None,with_proba=False)
x=sur.build_visualcomparison_table(inputs=res['ix_source'].values,
                                   targets=res['ix_target'].values,
                                   display=['name',],fuzzy=['street'],
                                   exact=['duns'])
x.sample(3)

starting deduplication at 2018-02-13 16:28:44.553932
1 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.584294 s
2 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.354111 s
3 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.369066 s
4 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.62118 s
5 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.365401 s
6 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.367494 s
7 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.411254 s
8 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.418704 s
9 of 10 inputs records deduplicated | found 1 of None max possible matches | time elapsed 0.435261 s
10 of 10 inputs records deduplicated | 

,,name_source,name_target,duns_source,duns_target,duns_exactscore,street_source,street_target,street_fuzzyscore
ix_source,ix_target,,,,,,,,
6097f4c6-8515-41fb-b5e5-549c81140848,f704e1f0-b240-4461-a741-b41b5c30b476,bildungswerk der wirtschaft hamburg,bildungswerk der wirtschaft hamburg,None,None,0.0,10 kapstadtring,10 kapstadtring,1.0
5ff704ee-399e-4fbd-b604-51b6ced944dd,af8133f8-361f-494e-92dc-ab3c72637d56,berlinzeppelin,berlinzeppelin,None,None,0.0,4 rottweiler str,4 rottweiler str,1.0
e2a2da69-3aa4-44e2-ae5b-4bbd2cbdc238,c906f2e3-bd4c-4785-9d60-95f19579a04c,botschaft afghanistan,botschaft afghanistan,None,None,0.0,3 taunusstraaye,3 taunusstr,0.7
